In [24]:
import bs4
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
## Digital Ingestion
from langchain_community.document_loaders import TextLoader, WebBaseLoader, PyPDFLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma, FAISS

load_dotenv()

## LOAD TEXT FILE
loader = TextLoader("../files/speech.txt")
text_document = loader.load()

## LOAD WEB BASED DOC.  Load, chunk and index the content of the html page
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                           class_=("post-title", "post-content", "post-header")  ## html tags classes you want to read
                       )))
web_documents = loader.load()

## LOAD PDF file
loader = PyPDFLoader("../files/attention.pdf")
pdf_document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(pdf_document)


In [25]:
## DB : Vector DB, Embedding and Vector Store
db_chroma = Chroma.from_documents(documents[:20], OpenAIEmbeddings())
db_faiss = FAISS.from_documents(documents[:20],OpenAIEmbeddings())

In [26]:
# query = "Who are the authors of the attentions is all you need research paper"
# query = "What is attentions is all you need"
# query = "An attention function can be described as mapping query"
# results_chroma = db_chroma.similarity_search(query)
# results_chroma[0].page_content
#
query = "An attention function can be described as mapping query"
results_faiss = db_faiss.similarity_search(query)
results_faiss[0].page_content

'3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3'